Exercise - Add Memory and Self-reflection - STARTER
In this exercise, you’ll enhance your AI agent by adding self-reflection and memory. These features allow the agent to iteratively critique its responses and improve over time while maintaining a log of all interactions.

This mimics how human learning and feedback loops work, pushing your agent towards more refined and accurate outputs.

Challenge

You are tasked with upgrading the existing agent. This version can learn from its previous answers, identify mistakes, and refine its responses automatically.

0. Import the necessary libs

In [1]:
import json
from typing import List, Dict, Literal
from openai import OpenAI
from openai.types.chat.chat_completion_message import ChatCompletionMessage

1. Recap: how to use OpenAI client with your API Key
To be able to connect with OpenAI, you need to instantiate an OpenAI client passing your OpenAI key.

You can pass the api_key argument directly.

client = OpenAI(api_key="voc-")

In [2]:
## TODO - Instantiate your client
## in a cell
#client = OpenAI(
#    api_key = "YOUR_API_KEY_HERE"
#)
##or using venv variables
#from dotenv import load_dotenv
#load_dotenv()
#client = OpenAI()

from openai import OpenAI
client = OpenAI()  # reads OPENAI_API_KEY from env

# Auth + connectivity check
models = client.models.list()
print("Model count:", len(models.data))

Model count: 84


In [3]:
response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Answer all user questions"},
            {"role": "user", "content":"What have I asked?"},
        ],
        temperature=0.0,
    )
response.choices[0].message.content

"I'm unable to access previous interactions or remember past conversations. However, I'm here to help with any questions or topics you'd like to discuss now! What would you like to know?"

2. Recap: Adding Memory
In order to add reflection, you need to make sure your agent can keep track of all interactions. Let's quickly recap how to do it with a simple list.

In [5]:
memory = [
    {"role": "system", "content": "Answer all user questions"},
    {"role": "user", "content": "What's an API"},
]

In [6]:
new_response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=memory,
    temperature=0.0,
)

memory.append(
    {"role": "assistant", "content": new_response.choices[0].message.content}
)

memory

[{'role': 'system', 'content': 'Answer all user questions'},
 {'role': 'user', 'content': "What's an API"},
 {'role': 'assistant',
  'content': 'An API, or Application Programming Interface, is a set of rules and protocols that allows different software applications to communicate with each other. It defines the methods and data formats that applications can use to request and exchange information. APIs enable developers to access the functionality of other software services, libraries, or platforms without needing to understand their internal workings.\n\nAPIs can be used for various purposes, such as:\n\n1. **Web APIs**: Allow communication between web servers and clients, often using HTTP requests. Examples include RESTful APIs and GraphQL APIs.\n2. **Library APIs**: Provide a set of functions and procedures for developers to use in their applications, often found in programming libraries.\n3. **Operating System APIs**: Allow applications to interact with the operating system, such 

In [7]:
memory.append(
    {"role": "user", "content": "What have I asked?"}
)

memory

[{'role': 'system', 'content': 'Answer all user questions'},
 {'role': 'user', 'content': "What's an API"},
 {'role': 'assistant',
  'content': 'An API, or Application Programming Interface, is a set of rules and protocols that allows different software applications to communicate with each other. It defines the methods and data formats that applications can use to request and exchange information. APIs enable developers to access the functionality of other software services, libraries, or platforms without needing to understand their internal workings.\n\nAPIs can be used for various purposes, such as:\n\n1. **Web APIs**: Allow communication between web servers and clients, often using HTTP requests. Examples include RESTful APIs and GraphQL APIs.\n2. **Library APIs**: Provide a set of functions and procedures for developers to use in their applications, often found in programming libraries.\n3. **Operating System APIs**: Allow applications to interact with the operating system, such 

In [8]:
new_response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=memory,
    temperature=0.0,
)

memory.append(
    {"role": "assistant", "content": new_response.choices[0].message.content}
)

memory

[{'role': 'system', 'content': 'Answer all user questions'},
 {'role': 'user', 'content': "What's an API"},
 {'role': 'assistant',
  'content': 'An API, or Application Programming Interface, is a set of rules and protocols that allows different software applications to communicate with each other. It defines the methods and data formats that applications can use to request and exchange information. APIs enable developers to access the functionality of other software services, libraries, or platforms without needing to understand their internal workings.\n\nAPIs can be used for various purposes, such as:\n\n1. **Web APIs**: Allow communication between web servers and clients, often using HTTP requests. Examples include RESTful APIs and GraphQL APIs.\n2. **Library APIs**: Provide a set of functions and procedures for developers to use in their applications, often found in programming libraries.\n3. **Operating System APIs**: Allow applications to interact with the operating system, such 

3. Create a memory layer
Now that you remember how to use a list of messages, it's recommended to have a proper class to deal with more complicated cases.

Create the Memory class and add the following methods to it:

add_message
get_messages
last_message

In [10]:
# memory layer

class Memory:
    def __init__(self):
        self._messages: List[Dict[str, str]] = []

    def add_message(self, role: Literal['user', 'system', 'assistant'], content: str):
        self._messages.append({
            "role": role,
            "content": content
        })

    def get_messages(self) -> List[Dict[str, str]]:
        return self._messages

    # A new method
    def last_message(self) -> None:
        if self._messages:
            return self._messages[-1]

4. Update the Agent class
In this exercise, you will enhance the AI Agent with self-reflection capabilities, allowing it to critique its own responses and refine them iteratively. This feature enables the agent to evaluate its output and improve the response quality before delivering a final answer.

Objective

Your task is to modify the agent so that it can:

Store conversation history – Implement a memory mechanism to track interactions.
Generate an initial response – Process user input and return a response using the language model.
Critique its own response when enabled – If self-reflection is activated, the agent should generate feedback on its own answer.
Refine its response iteratively – Based on the self-critique, the agent should adjust its reply, improving clarity, accuracy, and relevance.
Steps

Implement a memory layer to retain conversation history.
Introduce a self-reflection mechanism that allows the agent to analyze its response and refine it.
Limit the number of self-reflection iterations to prevent excessive loops (minimum 1, maximum 3).
Ensure flexibility by allowing users to toggle self-reflection on or off.
Considerations

The agent should always generate at least one response before self-reflection.
If self-reflection is enabled, it should run at least once more to critique and improve its output.
The number of iterations should be controlled and not exceed three refinements.
Implement logging functionality (verbose mode) to track the refinement process.
Invoke

Refactor invoke() method. This method now should include:

self_reflection paramenter (default: False);
max_iter parameter (default: 1);
If self_reflection is set to True, it should use a loop to generate an initial response. Then critiquing and refining the response in subsequent iterations up to the number of iterations defined in max_iter.

Use the self.memory to store each step.

Rules for self-reflection:

Don't allow values less than 1
Don't allow values greater than 3
Max iter is controlled by self_reflection flag.
If set to true, it needs to call the LLM at least once more for the criticism
Your self critique prompt should start with something like: Reflect on your previous response. Extend it to make sure it identifies errors and provides a revised version.

In [11]:

SELF_CRITIQUE_PROMPT = """
Reflect on your previous response...
Identify any mistakes, areas for improvement, or ways to clarify the answer, making it more concise. 
Provide a revised response if necessary in a Json Output structure:
{
    "original_response": "",
    "revisions_needed": "",
    "updated_response": ""
}
"""

In [12]:
class Agent:
    """A self-reflection AI Agent"""

    def __init__(
        self,
        name:str = "Agent", 
        role:str = "Personal Assistant",
        instructions:str = "Help users with any question",
        model:str = "gpt-4o-mini",
        temperature:float = 0.0,
    ):
        self.name = name
        self.role = role
        self.instructions = instructions
        self.model = model
        self.temperature = temperature

        self.client = OpenAI()

        self.memory = Memory()
        self.memory.add_message(
            role="system",
            content=f"You're an AI Agent, your role is {self.role}, " 
                    f"and you need to {self.instructions}",
        )

        self.critique_prompt = SELF_CRITIQUE_PROMPT
    
    def invoke(self, 
               user_message: str, 
               self_reflection: bool = False, 
               max_iter: int = 1, 
               verbose: bool = False) -> str:
    
        # Rules
        # - Don't allow values less than 1
        # - Don't allow values greater than 3
        # - Max iter is controlled by self_reflection flag. 
        # - If set to true, it needs to call the LLM at least once more for the criticism

        self.memory.add_message(
            role="user",
            content=user_message
        )
        if verbose:
            self._log_last_message()

        max_iter = max_iter if max_iter >= 1 else 1
        max_iter = max_iter if max_iter <= 3 else 3
        max_iter = max_iter if self_reflection else 0.5
        loops = 2 * max_iter

        for i in range(loops):
            ai_message = self._get_completion(
                messages = self.memory.get_messages()
            )

            self.memory.add_message(
                role = "assistant",
                content = ai_message.content,
            )
            if verbose:
                self._log_last_message()

            if i < loops - 1:
                self.memory.add_message(
                    role = "user", 
                    content = self.critique_prompt
                )
                if verbose:
                    self._log_last_message()

                ai_message = self._get_completion(
                    messages = self.memory.get_messages()
                )

    def _get_completion(self, messages:List[Dict])-> ChatCompletionMessage:
        response = self.client.chat.completions.create(
            model=self.model,
            temperature=self.temperature,
            messages=messages
        )
        
        return response.choices[0].message

    def _log_last_message(self):
        print(f"### {self.memory.last_message()['role']} message ###\n".upper())
        print(f"{self.memory.last_message()['content']} \n")
        print("\n________________________________________________________________\n")


5. Build some agents and have fun
Create some specific agents and invoke them with self_reflection = True

In [13]:
agent = Agent()
agent.invoke(
    user_message="Pick only one. Who is the best character in Game of Thrones?",
    self_reflection=True,
    verbose=True,
)

### USER MESSAGE ###

Pick only one. Who is the best character in Game of Thrones? 


________________________________________________________________

### ASSISTANT MESSAGE ###

Choosing the best character in "Game of Thrones" is subjective and depends on personal preference. However, many fans often cite Tyrion Lannister as a standout character due to his wit, intelligence, and complex moral compass. He navigates the treacherous political landscape with a unique perspective and often provides some of the show's most memorable lines. Who is your favorite character? 


________________________________________________________________

### USER MESSAGE ###


Reflect on your previous response...
Identify any mistakes, areas for improvement, or ways to clarify the answer, making it more concise. 
Provide a revised response if necessary in a Json Output structure:
{
    "original_response": "",
    "revisions_needed": "",
    "updated_response": ""
}
 


____________________________________

In [14]:
agent.memory.get_messages()

[{'role': 'system',
  'content': "You're an AI Agent, your role is Personal Assistant, and you need to Help users with any question"},
 {'role': 'user',
  'content': 'Pick only one. Who is the best character in Game of Thrones?'},
 {'role': 'assistant',
  'content': 'Choosing the best character in "Game of Thrones" is subjective and depends on personal preference. However, many fans often cite Tyrion Lannister as a standout character due to his wit, intelligence, and complex moral compass. He navigates the treacherous political landscape with a unique perspective and often provides some of the show\'s most memorable lines. Who is your favorite character?'},
 {'role': 'user',
  'content': '\nReflect on your previous response...\nIdentify any mistakes, areas for improvement, or ways to clarify the answer, making it more concise. \nProvide a revised response if necessary in a Json Output structure:\n{\n    "original_response": "",\n    "revisions_needed": "",\n    "updated_response": ""\n

In [15]:
json.loads(agent.memory.last_message()["content"])["updated_response"]

"Many fans consider Tyrion Lannister the best character in 'Game of Thrones' due to his wit, intelligence, and ability to navigate the complex political landscape. His memorable lines and moral complexity make him a standout."

6. Experiment
Now that you understood how it works, experiment with new things.

Experiment new critique prompts
What happens when you increase the number of iterations?
Try adding an argument to invoke() method to inspect it (verbose=True)
What else can you try?